In [4]:
import networkx as nx 
import y0
from y0.graph import NxMixedGraph
import pandas as pd
import sympy as sy


# calibrate model
def calibrate_model(model:NxMixedGraph, dataset:pd.DataFrame)->dict[sy.Symbol,float]:
    """
    Calibrates the given model using the provided dataset.

    Parameters:
    - model (NxMixedGraph): The model to be calibrated, represented as a directed graph.
    - dataset (pd.DataFrame): The dataset containing the observed values for the variables in the model.

    Returns:
    - dict[sy.Expr, float]: A dictionary mapping symbolic expressions to the calibrated values.

    """

    return {}


def test_calibrate_model():
    
    test_model = NxMixedGraph.from_edges(directed=[('A','B'),('B','C'),('D','C')])

    test_data_dict = {
        'A': [1,2,3],
        'B': [-1,-2,-3],
        'C': [0,0,0],
        'D': [1,3,5.0]
    }
    test_data = pd.DataFrame(test_data_dict)
    expected_parameters = {}
    actual_parameters = calibrate_model(test_model,test_data)

    assert isinstance(actual_parameters, dict), "Expected output to be a dictionary"


test_calibrate_model()
    


# train 


In [5]:
def intervene_on_state_variable(model:NxMixedGraph, intervention:dict[y0.dsl.Variable,float])->NxMixedGraph:
    pass

def intervene_on_parameter(model:NxMixedGraph, intervention:dict[sy.Symbol,float])->NxMixedGraph:
    pass




In [6]:
def sample_model(model:NxMixedGraph, n_samples:int, parameters:dict[sy.Symbol,float], parameter_interventions, state_interventions)->pd.DataFrame:
    pass

In [7]:
def compute_average_treatment_effect(treatment1:pd.DataFrame, treatment2:pd.DataFrame, outcome_variables:list[str])->dict[str,float]:
    pass